In [ ]:
import torch.optim as optim
import torchlensmaker as tlm

In [ ]:

# Demo using surface anchors for surfaces

lens_radius = 60. # lens radius in mm
surface1 = tlm.CircularArc(lens_radius, [-200.])
surface2 = surface1.share(scale=-1.0)

optics = tlm.OpticalStack([
    tlm.ParallelBeamUniform(radius=0.9*lens_radius),
    tlm.FixedGap(20.),
    
    tlm.RefractiveSurface(surface1, (1.0, 1.49), anchors=(tlm.Anchor.Center, tlm.Anchor.Edge)),
    tlm.FixedGap(0.1),
    tlm.RefractiveSurface(surface2, (1.49, 1.0), anchors=(tlm.Anchor.Edge, tlm.Anchor.Center)),

    tlm.FixedGap(0.1),
    
    tlm.RefractiveSurface(surface1, (1.0, 1.49), anchors=(tlm.Anchor.Center, tlm.Anchor.Edge)),
    tlm.FixedGap(0.1),
    tlm.RefractiveSurface(surface2, (1.49, 1.0), anchors=(tlm.Anchor.Edge, tlm.Anchor.Center)),
    
    tlm.FixedGap(200.0), # focal length
    tlm.FocalPointLoss(),
])


tlm.render(optics, num_rays=10)

tlm.optimize(
    optics,
    optimizer = optim.Adam(optics.parameters(), lr=1e-3),
    num_rays = 20,
    num_iter = 150
)

tlm.render(optics, num_rays=10)